In [423]:
import os
import wave
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import seaborn as sns
from glob import glob
from obspy.core import read
from obspy import signal
import obspy
import obspy.signal
import librosa as lr
import plotly.graph_objs as go
import plotly.express as px
from subprocess import check_output
from scipy.signal import hilbert
%matplotlib inline

from random import random
import random
from array import *
import time

In [424]:
size = 1000
yy = np.random.rand(size)
xx = np.arange(size)
np.sort(yy) 
count = 0
check = 0
while(count<size):
    yy[count] += count
    hold = (random.uniform(-2.5,3.5)) 
    yy[count] = hold
    check +=1
    count +=6
    
#print(check)

randSpot = int(size-size/10)
#print(randSpot)
op = size / 25
while(randSpot < (size-size/10)+op):
    yy[randSpot] += 18
    randSpot+=1
    
#print(yy)
#print(op)
origGraph = [1] * size
x = 0
while (x < size) :
    origGraph[x] = yy[x]
    x += 1



In [425]:
fig = go.Figure(data=go.Scatter(x=xx, y=yy, mode='markers'))

fig.show()

In [426]:
from scipy.optimize import curve_fit

timeStart = time.time()
print("Seconds since epoch =", timeStart, "\n")	

def func(x, a, b):
    return a * x + b


p, pcov = curve_fit(func, xx, yy)
y_exp1 = func(xx, *p)
print(p)


fig = go.Figure(data = go.Scatter(x = xx, y=yy,
    name='Original Plot', mode='markers'))
fig.add_trace(go.Scatter(
    x=xx,
    y=y_exp1,
    mode='markers',
    marker=dict(
        color='black',
        size=5
    ),
    name='Linear'
))
fig.update_layout(
      xaxis=dict(
        title='Time (s)'
    ),
    yaxis=dict(
        title='Position in X'
    ))
fig.show()


Seconds since epoch = 1588739466.295766 

[ 0.00367731 -0.61729994]


In [427]:
count = 0
while(count<size):
    diff = y_exp1[count] - yy[count]
    if abs(diff) > 1.5:
        #print(diff)
        yy[count] = y_exp1[count]
    count +=1
    
fig = go.Figure()
fig.add_trace(go.Scatter(x = xx, y=yy,
    name='Original Plot', mode='markers'))
fig.add_trace(go.Scatter(x = xx, y=origGraph,
    name='Original Plot', mode='markers'))

In [428]:
from scipy.optimize import curve_fit

def func(x, a, b):
    return a * x + b


p, pcov = curve_fit(func, xx, yy)
y_exp = func(xx, *p)
print(p)


fig = go.Figure()
fig.add_trace(go.Scatter(x = xx, y=origGraph,
    name='Original Plot', mode='markers'))
fig.add_trace(go.Scatter(
    x=xx,
    y=y_exp,
    mode='markers',
    marker=dict(
        color='black',
        size=5
    ),
    name='Adjusted'
))
fig.add_trace(go.Scatter(
    x=xx,
    y=y_exp1,
    mode='markers',
    marker=dict(
        color='red',
        size=5
    ),
    name='Old'
))
fig.update_layout(
      xaxis=dict(
        title='Time (s)'
    ),
    yaxis=dict(
        title='Position in X'
    ))
fig.show()

timeEnd = time.time()
timeElapsed = timeEnd - timeStart
#print("Seconds since epoch =", timeStart, "\n")	
#print("Time elapsed in seconds =", timeElapsed, "\n")	


[ 0.00272949 -0.26250117]


In [429]:
#-------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------

In [430]:
timeStart = time.time()
#print("Seconds since epoch =", timeStart, "\n")	

#Given:
#    data – A set of observations.
#    model – A model to explain observed data points.
#    n – Minimum number of data points required to estimate model parameters.
#    k – Maximum number of iterations allowed in the algorithm.
#    t – Threshold value to determine data points that are fit well by model.
#    d – Number of close data points required to assert that a model fits well to data.

from math import exp, expm1
import math
iterations = 0
bestFit = []
sampleSize = int(size / 3)
bestOutliers = 100

x_temp = np.arange(sampleSize)

def func(x, a, b):
    return a * x + b

i = 0
N = int(size / 3) # amount of samples taken

while(i < N):
    outliers = 0
    samples = 0
    set = [1] * sampleSize
    
    while(samples < sampleSize): # picks random points for micro graph
        samp = int((random.uniform(0,size))) 
        set[samples] = origGraph[samp]
        samples += 1
    
    p, pcov = curve_fit(func, x_temp, set) # fits a y = mx+b line to the selected points
    mini_y_exp = func(x_temp, *p)
    #print(set)
    
    count = 0
    while(count<sampleSize):
        diff = mini_y_exp[count] - set[count]
        if abs(diff) > 0.3:
            #print(diff)
            outliers += 1
        count +=1
    if outliers < bestOutliers:
        bestOutliers = outliers
        #print(outliers)
        bestLine = mini_y_exp
    
    i += 1
    
timeEnd = time.time()
timeElapsedRansac = timeEnd - timeStart
#print("Seconds since epoch =", timeStart, "\n")	
#print("Time elapsed in seconds =", timeElapsedRansac, "\n")	
#print(bestLine)

In [431]:
print(p) # a * x + b

fig = go.Figure()
fig.add_trace(go.Scatter(x = xx, y=origGraph,
    name='Original Plot', mode='markers'))
fig.add_trace(go.Scatter(
    x=xx,
    y = (p[0] * xx + p[1]),
    #y=mini_y_exp,
    mode='markers',
    marker=dict(
        color='black',
        size=5
    ),
    name='RANSAC Algorithm'
))
fig.add_trace(go.Scatter(
    x=xx,
    y=y_exp,
    mode='markers',
    marker=dict(
        color='red',
        size=5
    ),
    name='Trivial Solution'
))
fig.add_trace(go.Scatter(
    x=xx,
    y=y_exp1,
    mode='markers',
    marker=dict(
        color='cyan',
        size=2
    ),
    name='No change linear regression'
))

fig.show()

print("Time elapsed in seconds for RANSAC =", timeElapsedRansac, "\n")	
print("Time elapsed in seconds for Trivial Solution=", timeElapsed, "\n")	

[4.97091958e-04 1.19927187e+00]


Time elapsed in seconds for RANSAC = 0.44699883460998535 

Time elapsed in seconds for Trivial Solution= 0.5589995384216309 

